In [1]:
from scripts import setup_environment

setup_environment()

In [2]:
import textwrap

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# System message templates (priming)
system_templates = {
    "default": "You are an AI assistant.",
    "default_improved": "You are an AI assistant specialized in solving lateral thinking questions. You will receive a question with multiple choices and must determine the correct answer using logical reasoning and problem-solving techniques.",
    "step_by_step": "You are a logical problem solver. Break down the question systematically, analyze each answer choice step by step, eliminate incorrect options, and select the best answer.",
    "creative": "You are a lateral thinker. Approach each question with flexible reasoning, exploring unconventional yet valid interpretations before selecting the best answer.",
    "elimination": "You are a strategic reasoner. First, identify and eliminate incorrect answer choices. Then, select the most logical remaining option.",
    "metaphor": "You are skilled in abstract reasoning. Consider both literal and metaphorical meanings in the question and choices before selecting the most insightful answer.",
    "confidence": "You are an analytical decision-maker. Assess the likelihood of correctness for each choice, score them internally, and select the answer with the highest confidence.",
    "perspective_shift": "You are a multi-perspective analyst. Evaluate the question from different angles, considering alternative interpretations before determining the best answer.",
    "common_sense": "You balance logic and practicality. Apply both structured reasoning and real-world common sense to determine the most reasonable answer.",
    "assumption_challenge": "You are a critical thinker. Identify and question hidden assumptions in the question and choices before selecting the answer that best challenges or aligns with them.",
    "pattern_matching": "You recognize patterns and relationships. Identify logical structures, recurring themes, or hidden connections in the question and choices before selecting the best answer.",
    "intuitive": "You combine intuition with logic. Generate an initial answer instinctively, then critically evaluate it for logical soundness before finalizing your choice.",
}


def get_system_prompt(template_name: str):
    system_prompt = system_templates[template_name]
    system_prompt = textwrap.dedent(system_prompt)

    system_prompt_template = SystemMessagePromptTemplate.from_template(
        system_prompt, id=template_name
    )
    return system_prompt_template


def get_user_prompt():
    prompt = """
    Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:

    Question: {question}
    Choice:
    {choices}
    Answer:
    """

    prompt = textwrap.dedent(prompt)

    prompt_template = HumanMessagePromptTemplate.from_template(prompt)
    return prompt_template


def create_prompt_template(
    system_prompt_template_name: str = "default",
):
    system_prompt_template = get_system_prompt(system_prompt_template_name)
    user_prompt_template = get_user_prompt()
    chat_prompt_template = ChatPromptTemplate.from_messages(
        [system_prompt_template, user_prompt_template]
    )

    return chat_prompt_template

In [3]:
from scripts.dataset import BrainteaserDataset

dataset = BrainteaserDataset("data")

In [4]:
import string

from scripts.dataset import RiddleQuestion


def args_generator(riddle_question: RiddleQuestion):
    template_args = {
        "question": riddle_question.question,
        "choices": "\n".join(
            [
                f"({string.ascii_uppercase[j]}) {choice}"
                for j, choice in enumerate(riddle_question.choice_list)
            ]
        ),
    }

    return template_args


chat_prompt_template = create_prompt_template("default")

In [5]:
from scripts.lmm import OllamaModel
from scripts.executor import Executor

base_url = "http://136.61.33.107:41302"
executor = Executor(
    models=[
        # Llama3.1
        OllamaModel("llama3.1:8b", base_url=base_url),
        # Llama3.2
        OllamaModel("llama3.2:1b", base_url=base_url),
        OllamaModel("llama3.2:3b", base_url=base_url),
        # Phi3.5
        OllamaModel("phi3.5:3.8b", base_url=base_url),
        # Phi4
        OllamaModel("phi4:14b", base_url=base_url),
        # Qwen2.5
        OllamaModel("qwen2.5:0.5b", base_url=base_url),
        OllamaModel("qwen2.5:1.5b", base_url=base_url),
        OllamaModel("qwen2.5:3b", base_url=base_url),
        OllamaModel("qwen2.5:7b", base_url=base_url),
        OllamaModel("qwen2.5:14b", base_url=base_url),
        OllamaModel("qwen2.5:32b", base_url=base_url),
        # Gemma2
        OllamaModel("gemma2:2b", base_url=base_url),
        OllamaModel("gemma2:9b", base_url=base_url),
        OllamaModel("gemma2:27b", base_url=base_url),
        # Mistral Nemo
        OllamaModel("mistral-nemo:12b", base_url=base_url),
    ]
)

2025-02-17 20:20:18,821 - INFO - Initialized executor with 15 models.


In [6]:
test_sample_size = 50
sp_data = dataset.sp[0:test_sample_size]
wp_data = dataset.wp[0:test_sample_size]

In [7]:
from pathlib import Path

import dill as pickle

total_results = {}

reduced_dataset = dataset.sp[0:]
for technique in system_templates:
    chat_prompt_template = create_prompt_template(technique)
    sp_results = await executor.aexecute(
        sp_data,
        chat_prompt_template,
        args_generator,
        dump_to_pickle=True,
        create_checkpoints=True,
        resume_from_checkpoint=True,
        result_file_name=f"test_0/sp_results_{technique}",
    )

    wp_results = await executor.aexecute(
        wp_data,
        chat_prompt_template,
        args_generator,
        dump_to_pickle=True,
        create_checkpoints=True,
        resume_from_checkpoint=True,
        result_file_name=f"test_0/wp_results_{technique}",
    )

    total_results[technique] = {"sp": sp_results, "wp": wp_results}

results_file = Path("test_0/results_test_0_system-prime-messages.pkl")
results_file.parent.mkdir(parents=True, exist_ok=True)

with results_file.open("wb") as f:
    pickle.dump(total_results, f)

2025-02-17 20:20:18,835 - INFO - Loading results from result file: results/test_0/sp_results_default.pkl
2025-02-17 20:20:18,930 - INFO - Results file is valid, returning results
2025-02-17 20:20:18,930 - INFO - Loading results from result file: results/test_0/wp_results_default.pkl
2025-02-17 20:20:18,938 - INFO - Results file is valid, returning results
2025-02-17 20:20:18,938 - INFO - Loading results from result file: results/test_0/sp_results_default_improved.pkl
2025-02-17 20:20:18,946 - INFO - Results file is valid, returning results
2025-02-17 20:20:18,946 - INFO - Loading results from result file: results/test_0/wp_results_default_improved.pkl
2025-02-17 20:20:18,955 - INFO - Results file is valid, returning results
2025-02-17 20:20:18,956 - INFO - Loading results from result file: results/test_0/sp_results_step_by_step.pkl
2025-02-17 20:20:19,049 - INFO - Results file is valid, returning results
2025-02-17 20:20:19,050 - INFO - Loading results from result file: results/test_0/

llama3.1:8b:   0%|          | 0/50 [00:00<?, ?it/s]

2025-02-17 20:20:41,991 - INFO - Creating checkpoint: results/checkpoints/test_0/sp_results_pattern_matching/llama3.1:8b_test_0/sp_results_pattern_matching.pkl
2025-02-17 20:20:42,002 - INFO - Cleaning up llama3.1:8b
2025-02-17 20:20:42,003 - INFO - Ollama models will be deleted on demand and therefore this step is skipped!
2025-02-17 20:20:42,003 - INFO - Processing llama3.2:1b
2025-02-17 20:20:42,003 - INFO - Pulling Ollama model: llama3.2:1b


llama3.2:1b:   0%|          | 0/50 [00:00<?, ?it/s]

2025-02-17 20:20:56,874 - INFO - Creating checkpoint: results/checkpoints/test_0/sp_results_pattern_matching/llama3.2:1b_test_0/sp_results_pattern_matching.pkl
2025-02-17 20:20:56,886 - INFO - Cleaning up llama3.2:1b
2025-02-17 20:20:56,886 - INFO - Ollama models will be deleted on demand and therefore this step is skipped!
2025-02-17 20:20:56,887 - INFO - Processing llama3.2:3b
2025-02-17 20:20:56,887 - INFO - Pulling Ollama model: llama3.2:3b


...

2025-02-17 21:13:32,613 - INFO - Creating checkpoint: results/checkpoints/test_0/wp_results_intuitive/gemma2:9b_test_0/wp_results_intuitive.pkl
2025-02-17 21:13:32,625 - INFO - Cleaning up gemma2:9b
2025-02-17 21:13:32,625 - INFO - Ollama models will be deleted on demand and therefore this step is skipped!
2025-02-17 21:13:32,626 - INFO - Processing gemma2:27b
2025-02-17 21:13:32,626 - INFO - Pulling Ollama model: gemma2:27b


gemma2:27b:   0%|          | 0/50 [00:00<?, ?it/s]

2025-02-17 21:15:26,140 - INFO - Creating checkpoint: results/checkpoints/test_0/wp_results_intuitive/gemma2:27b_test_0/wp_results_intuitive.pkl
2025-02-17 21:15:26,151 - INFO - Cleaning up gemma2:27b
2025-02-17 21:15:26,151 - INFO - Ollama models will be deleted on demand and therefore this step is skipped!
2025-02-17 21:15:26,152 - INFO - Processing mistral-nemo:12b
2025-02-17 21:15:26,152 - INFO - Pulling Ollama model: mistral-nemo:12b


mistral-nemo:12b:   0%|          | 0/50 [00:00<?, ?it/s]

2025-02-17 21:16:09,847 - INFO - Creating checkpoint: results/checkpoints/test_0/wp_results_intuitive/mistral-nemo:12b_test_0/wp_results_intuitive.pkl
2025-02-17 21:16:09,860 - INFO - Cleaning up mistral-nemo:12b
2025-02-17 21:16:09,860 - INFO - Ollama models will be deleted on demand and therefore this step is skipped!
2025-02-17 21:16:09,861 - INFO - Asynchronous execution complete
2025-02-17 21:16:09,861 - INFO - Dumping results to results/test_0/wp_results_intuitive.pkl
